In [ ]:
##
# @file This file is part of EDGE.
#
# @author Alexander Breuer (anbreuer AT ucsd.edu)
#
# @section LICENSE
# Copyright (c) 2016, Regents of the University of California
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:
#
# 1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
#
# 2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
#
# 3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
# @section DESCRIPTION
# Derivation of basis functions and the attached matrices for tetrahedral elements.
##

In [ ]:
import sympy
import matplotlib.pyplot
sympy.init_printing(use_unicode=True)

In [ ]:
# upper polynomial degree
l_degUp = 2

# basis
l_basis = []

xi1 = sympy.symbols('xi_1') # first tetrahedral coord
xi2 = sympy.symbols('xi_2') # second tetrahedral coord
xi3 = sympy.symbols('xi_3') # third tetrahedral coord

eta1 = sympy.symbols('eta_1') # first collapsed coord
eta2 = sympy.symbols('eta_2') # scond collapsed coord
eta3 = sympy.symbols('eta_3') # third collapsed coord

# derive basis functions
for l_order in xrange(0, l_degUp+1):
  for l_p1 in xrange(0, l_order+1):
    # first principal function
    l_psiA = sympy.jacobi( l_p1, 0, 0, eta1 )
        
    for l_p2 in xrange(0, l_order+1):
      # second principal function
      l_psiB = ( ( (1-eta2)/2 )**l_p1 * sympy.jacobi( l_p2, 2*l_p1+1, 0, eta2 ) )

      for l_p3 in xrange(0, l_order+1):
        if( l_p1 + l_p2 + l_p3 == l_order): # build hierarchical basis following pascals triangle
          # third principal function
          l_psiC = ( (1-eta3)/2 )**(l_p1 + l_p2) * sympy.jacobi( l_p3, 2*l_p1+2*l_p2+2, 0, eta3 )

          l_basis = l_basis + [l_psiA * l_psiB * l_psiC]
        
          # insert tetrahedral coordinates
          l_basis[-1] = l_basis[-1].subs( eta1, 2*(1+xi1) / (-xi2-xi3) - 1 )
          l_basis[-1] = l_basis[-1].subs( eta2, 2*(1+xi2) / (1-xi3)    - 1 )
          l_basis[-1] = l_basis[-1].subs( eta3, xi3 )

          # use tets with xi1, xi2, xii3 in [0,1]
          l_basis[-1] = l_basis[-1].subs(xi1, 2*xi1-1)
          l_basis[-1] = l_basis[-1].subs(xi2, 2*xi2-1)
          l_basis[-1] = l_basis[-1].subs(xi3, 2*xi3-1)
        
          # simplify the basis
          l_basis[-1] = sympy.simplify( l_basis[-1] )

In [ ]:
# generate code for the basis
print "assert( b <", len(l_basis), ");"
for l_b in xrange(0, len(l_basis)):
  print "if( b ==", l_b, ") {" 
  print "  val =", sympy.ccode( l_basis[l_b]  ).replace("pow", "std::pow"), ";"
  print "}"

# generate code for the derivatives
print '*******************************'
print "assert( b <", len(l_basis), ");"
for l_b in xrange(0, len(l_basis)):
  print "if( b ==", l_b, ") {" 
  print "  valDxi1 =", sympy.ccode( sympy.diff(l_basis[l_b], xi1)  ).replace("pow", "std::pow"), ";"
  print "}"
print '*******************************'
print "assert( b <", len(l_basis), ");"
for l_b in xrange(0, len(l_basis)):
  print "if( b ==", l_b, ") {" 
  print "  valDxi2 =", sympy.ccode( sympy.diff(l_basis[l_b], xi2)  ).replace("pow", "std::pow"), ";"
  print "}"
print '*******************************'
print "assert( b <", len(l_basis), ");"
for l_b in xrange(0, len(l_basis)):
  print "if( b ==", l_b, ") {" 
  print "  valDxi3 =", sympy.ccode( sympy.diff(l_basis[l_b], xi3)  ).replace("pow", "std::pow"), ";"
  print "}"

In [ ]:
# setup functions for upcoming matrix computations
l_fInt    = lambda bf: sympy.integrate( bf, (xi1, 0, 1-xi2-xi3), (xi2, 0, 1-xi3), (xi3, 0, 1) )
l_diffXi1 = lambda bf: sympy.diff( bf, xi1)
l_diffXi2 = lambda bf: sympy.diff( bf, xi2)
l_diffXi3 = lambda bf: sympy.diff( bf, xi3 )


# compute mass matrix
l_mm = sympy.Matrix( l_basis ) * sympy.Matrix.transpose( sympy.Matrix( l_basis ) )
l_mm = l_mm.applyfunc( l_fInt )

# compute stiffness matrices
l_sm1 = sympy.Matrix.transpose( sympy.Matrix( l_basis ) )
l_sm1 = l_sm1.applyfunc( l_diffXi1 )
l_sm1 = sympy.Matrix( l_basis ) * l_sm1
l_sm1 = l_sm1.applyfunc( l_fInt )

l_sm2 = sympy.Matrix.transpose( sympy.Matrix( l_basis ) )
l_sm2 = l_sm2.applyfunc( l_diffXi2 )
l_sm2 = sympy.Matrix( l_basis ) * l_sm2
l_sm2 = l_sm2.applyfunc( l_fInt )

l_sm3 = sympy.Matrix.transpose( sympy.Matrix( l_basis ) )
l_sm3 = l_sm3.applyfunc( l_diffXi3 )
l_sm3 = sympy.Matrix( l_basis ) * l_sm3
l_sm3 = l_sm3.applyfunc( l_fInt )

In [ ]:
# compute flux matrices
#
#   face 0: 0-1-2
#   face 1: 0-3-1
#   face 2: 0-2-3
#   face 3: 1-3-2
#                 zeta
#                  3: x
#                   *
#                  *    *
#                 *   *
#                *         *
#               *
#              *       *      *
#             *
#            *                    *
#           * origin(0): x
#          *           *     *       *
#         *        *              *
#        *      *                       x
#       *    *               *          2: eta
#      *  *     *
# 1: x
# xi
#

l_localFm = dict()
l_neighFm = dict()

chi1 = sympy.symbols('chi_1')
chi2 = sympy.symbols('chi_2')
hchi1 = sympy.symbols('alpha_1')
hchi2 = sympy.symbols('alpha_2')


# define mapping functions from face coords to volume coords
# and
# from one face to face
#
# Reference: Dumbser, Michael, and Martin Käser.
#            An arbitrary high-order discontinuous Galerkin method for elastic waves on unstructured meshes—II.
#            The three-dimensional isotropic case." Geophysical Journal International 167.1 (2006): 319-336.
#            Table 2
l_faToVol = []
l_faToVol = l_faToVol + [ lambda bf: bf.subs(xi1, chi2        ).subs( xi2, chi1 ).subs( xi3, 0    ) ]
l_faToVol = l_faToVol + [ lambda bf: bf.subs(xi1, chi1        ).subs( xi2, 0    ).subs( xi3, chi2 ) ]
l_faToVol = l_faToVol + [ lambda bf: bf.subs(xi1, 0           ).subs( xi2, chi2 ).subs( xi3, chi1 ) ]
l_faToVol = l_faToVol + [ lambda bf: bf.subs(xi1, 1-chi1-chi2 ).subs( xi2, chi1 ).subs( xi3, chi2 ) ]

l_faToFa = []
l_faToFa = l_faToFa + [ lambda bf: bf.subs(hchi1, chi2         ).subs( hchi2, chi1         ) ]
l_faToFa = l_faToFa + [ lambda bf: bf.subs(hchi1, 1-chi1-chi2  ).subs( hchi2, chi2         ) ]
l_faToFa = l_faToFa + [ lambda bf: bf.subs(hchi1, chi1         ).subs( hchi2, 1-chi1-chi2 ) ]



# define integration in terms of face coords
l_fluxInt = lambda bf: sympy.integrate( bf, (chi1, 0, 1-chi2), (chi2, 0, 1) )

# local face
for l_f1 in xrange(0,4):
  # determine basis
  l_basisF1 = []
  for l_bf in xrange(0, len(l_basis)):
    if l_basis[l_bf].is_constant() == False:
      # replace volume with face coords if not constant
      l_basisF1 = l_basisF1 + [ l_faToVol[l_f1]( l_basis[l_bf] ) ]
    else:
      l_basisF1 = l_basisF1 + [ l_basis[l_bf] ]

  # assemnle flux matrix
  l_localFm['fm'+str(l_f1)] = sympy.Matrix( l_basisF1 ) * \
                              sympy.Matrix.transpose( sympy.Matrix( l_basisF1 ))
  # do the integration
  l_localFm['fm'+str(l_f1)] = l_localFm['fm'+str(l_f1)].applyfunc( l_fluxInt )
    
  # neighboring face
  for l_f2 in xrange(0,4):
    # neighboring face-vertex
    for l_ve in xrange(0,3):
      l_basisF2 = []
        
      for l_bf in xrange(0, len(l_basis)):
        if l_basis[l_bf].is_constant() == False:
          # replace volume with face coords if not constant
          l_basisF2 = l_basisF2 + [ l_faToVol[l_f2]( l_basis[l_bf] ) ]
        else:
          l_basisF2 = l_basisF2 + [ l_basis[l_bf] ]
        
        # change name of face parameters
        l_basisF2[-1] = l_basisF2[-1].subs( chi1, hchi1 ).subs( chi2, hchi2)

        # insert first face's parameters
        l_basisF2[-1] = l_faToFa[l_ve]( l_basisF2[-1] )
        
      # assemble the flux matrix
      l_neighFm['fp'+str(l_f1)+str(l_f2)+str(l_ve)] = sympy.Matrix( l_basisF2 ) * \
                                                      sympy.Matrix.transpose( sympy.Matrix( l_basisF1 ))
            
      # do the integration
      l_neighFm['fp'+str(l_f1)+str(l_f2)+str(l_ve)] = l_neighFm['fp'+str(l_f1)+str(l_f2)+str(l_ve)].applyfunc( l_fluxInt )

In [ ]:
l_mm, l_sm1, l_sm2, l_sm3, l_localFm, l_neighFm